<img src="https://raw.githubusercontent.com/Databricks-BR/lab_sql/main/images/header_handson_sql.png">

###Objetivo: Criar uma Tabela que contém informações sensíveis de contato e tratar utilizando uma função escrita em Python

##1. Criação da tabela
```sql
CREATE OR REPLACE TABLE <catalog>.<schema>.contatos_bronze
  ( user_id      INT    COMMENT "codigo do usuario",
    contato STRING COMMENT "dados de contato do usuario" )
COMMENT "Tabela com informacoes de contato dos usuarios";
```

##2. Inserindo informações de contato

```SQL
INSERT INTO <catalog>.<schema>.contatos_bronze (user_id, contato) VALUES (1, '{
  "name": "Joao Silva",
  "email": "joao@example.com",
  "phone": "+55 11 99999-9999",
  "birthdate": "1985-01-01"
}');
INSERT INTO <catalog>.<schema>.contatos_bronze (user_id, contato) VALUES (2, '{
  "name": "Maria Dias",
  "email": "maria@example.com",
  "phone": "+55 11 88888-8888",
  "birthdate": "1990-05-15"
}');
INSERT INTO <catalog>.<schema>.contatos_bronze (user_id, contato) VALUES (3, '{
  "name": "Jose Santos",
  "email": "jose@example.com",
  "phone": "+55 11 77777-7777",
  "birthdate": "1983-11-20"
}');
INSERT INTO <catalog>.<schema>.contatos_bronze (user_id, contato) VALUES (4, '{
  "name": "Ana Marques",
  "email": "ana@example.com",
  "phone": "+55 11 66666-6666",
  "birthdate": "1995-02-28"
}');
INSERT INTO <catalog>.<schema>.contatos_bronze (user_id, contato) VALUES (5, '{
  "name": "Pedro Matos",
  "email": "pedro@example.com",
  "phone": "+55 11 55555-5555",
  "birthdate": "1978-09-10"
}');
```

##3.Verificando valores
```sql
SELECT * 
FROM <catalog>.<schema>.contatos_bronze;
```

##4.Criar função de redact
```sql
CREATE OR REPLACE FUNCTION <catalog>.<schema>.redact_field(a STRING)
RETURNS STRING
LANGUAGE PYTHON
AS $$
import json
keys = ["email", "phone", "birthdate"]
obj = json.loads(a)
for k in obj:
  if k in keys:
    obj[k] = "REDACTED"
return json.dumps(obj)
$$;
```

##5. Testar função
```sql
SELECT 
    user_id as id, 
    <catalog>.<schema>.redact_field(contato) as contato
FROM <catalog>.<schema>.contatos_bronze;
```

##6. Criar VIEW para consumo
```sql
CREATE OR REPLACE VIEW <catalog>.<schema>.contatos_redacted AS
WITH redact_contatos AS (
    SELECT 
        user_id, 
        <catalog>.<schema>.redact_field(contato) as contato
    FROM <catalog>.<schema>.contatos_bronze
)
SELECT
    user_id,
    contato:name as name,
    contato:phone as phone,
    contato:email as email,
    contato:birthdate as birthdate
FROM
    redact_contatos;
```

##7. Veriricar valores redacted
```sql
SELECT *
FROM <catalog>.<schema>.contatos_redacted;
```